<a href="https://colab.research.google.com/github/joelwwiggins/car_ml/blob/main/Copy_of_notebook_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [5]:
#import csv from path
#drop first row and column
df1 = pd.read_csv('CSVLog_20221130_091051.csv', skiprows=1)


In [6]:
#drop first column
df1_col=df1.iloc[:,1:]
df=df1_col

In [7]:
df.head()

,Latitude (deg),Longitude (deg),Vehicle speed (MPH),Instant fuel economy (MPG),Total fuel economy (MPG),Fuel rate (gal/hr),Instant CO2 rate (lb/mile),Total CO2 (lbs),CO2 flow (lb/min),Accel X (ft/s²),...,Average Trip CO2 Rate (lb/mile),Max Speed (MPH),Seconds Idling (sec),Idling Count,Hard Accel Count,Hard Brake Count,Trip Duration (min),Trip Fuel Economy (MPG),Trip Fuel (gal),Trip Distance (miles)
0,29.56902,-95.21054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0
1,29.56902,-95.21054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.294891,...,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0
2,29.56902,-95.21054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.317437,...,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0
3,29.56902,-95.21054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.317437,...,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0
4,29.56902,-95.21054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.346312,...,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0


In [8]:
#convert to numpy array
df_np=df.to_numpy()

#train test split
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df_np, test_size=0.2, random_state=42)

#normalize the x_train
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

#reshape the data to 3d
X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1],1))

#use a 1dconv model to predict the next value
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D 
from keras.layers import Dropout
from keras.layers import LSTM

model_shape = X_train_scaled.shape



nn_model = tf.keras.models.Sequential()
nn_model.build(input_shape=model_shape)

nn_model.add(tf.keras.layers.Conv1D(512,3,activation="relu",padding='same',input_shape=model_shape[1:]))

nn_model.add(tf.keras.layers.MaxPool1D(pool_size=2,data_format='channels_last',padding='same'))


nn_model.add(tf.keras.layers.Flatten())
nn_model.add(tf.keras.layers.Dense(units=model_shape[1], activation='linear'))




nn_model.compile(loss='mse', optimizer='adam', metrics=['mae','mse'])
es = tf.keras.callbacks.EarlyStopping(monitor='val_mae',patience=10)
# checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath ="/content/sample_data")
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=1, write_graph=True, write_images=True)
cb_list = [es, tbCallBack]
nn_model.fit(X_train, X_train, epochs=30,validation_split=.1,shuffle=True,callbacks=cb_list)
#normalize the x_test
X_test_scaled = scaler.fit_transform(X_test)
nn_model.evaluate(X_test_scaled, X_test_scaled)



nn_model.summary()

Epoch 1/30
845/845 [==============================] - 14s 6ms/step - loss: 204440.6562 - mae: 33.4144 - mse: 204440.6562 - val_loss: 673.4474 - val_mae: 18.0554 - val_mse: 673.4474
Epoch 2/30
845/845 [==============================] - 5s 6ms/step - loss: 1268.3369 - mae: 25.4572 - mse: 1268.3369 - val_loss: 1377.4548 - val_mae: 27.9250 - val_mse: 1377.4548
Epoch 3/30
845/845 [==============================] - 5s 6ms/step - loss: 1072.1978 - mae: 23.7671 - mse: 1072.1978 - val_loss: 394.0660 - val_mae: 14.5517 - val_mse: 394.0660
Epoch 4/30
845/845 [==============================] - 5s 6ms/step - loss: 865.0702 - mae: 20.8605 - mse: 865.0702 - val_loss: 1065.8761 - val_mae: 21.4992 - val_mse: 1065.8761
Epoch 5/30
845/845 [==============================] - 5s 6ms/step - loss: 660.8025 - mae: 18.5507 - mse: 660.8025 - val_loss: 192.8486 - val_mae: 10.9094 - val_mse: 192.8486
Epoch 6/30
845/845 [==============================] - 5s 6ms/step - loss: 531.2279 - mae: 16.3362 - mse: 531.2279 -

In [10]:
nn_model.save(. /)

SyntaxError: ignored

In [1]:


#plot the loss
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

#predict the next value
X_test_scaled = scaler.transform(X_test)
yhat = model.predict(X_test_scaled)

#inverse transform the predicted value
yhat_inv = scaler.inverse_transform(yhat)

#inverse transform the actual value
X_test_inv = scaler.inverse_transform(X_test)

#plot the actual and predicted value
plt.plot(X_test_inv[:,0], label='actual')
plt.plot(yhat_inv[:,0], label='predicted')
plt.legend()
plt.show()



NameError: ignored